# x 차 년도 케이웨더 데이터 삭제 모듈
(2023.03.22, 최종 업데이트)

## 1. 초기 세팅 & DB 서버 연동

In [1]:
import pandas as pd
import os #기타 운영 체제 인터페이스
import sys #시스템 특정 파라미터와 함수
from datetime import datetime
sys.path.append("../")
sys.path.append("../..") #path는 모듈의 검색 경로를 지정하는 문자열 리스트

from pathlib import Path
UTC_Style = '%Y-%m-%dT%H:%M:%SZ'

In [2]:
from Clust.setting import influx_setting_KETI as ins
from influxdb_client.client.write_api import SYNCHRONOUS, ASYNCHRONOUS
from influxdb_client import InfluxDBClient, Point, BucketsService, Bucket

version = 2 # or 1

if version == 1: 
    from Clust.clust.ingestion.influx import influx_client as influx_Client
    dataServer = ins.CLUSTDataServer
elif version == 2:
    from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
    dataServer = ins.CLUSTDataServer2
    
InfluxClient = influx_Client.InfluxClient(dataServer)

## 2. 함수 구현

In [3]:
#################################################################
# 사용자가 잘못 저장한 measurement data를 db에서 실제 삭제
#################################################################
def drop_measurement_of_specific_time(folder_path, folder_name, file_name) :
    # (1) x차 년도 데이터가 실제 저장된 폴더를 읽어 각 measuremetn의 첫째 날짜 추출 
    file_path = os.path.join(folder_path, file_name)
    file_contents = pd.read_csv(file_path, encoding = 'UTF-8')

    # (2) 삭제 시 필요한  파라미터 초기화
    # ['out_시각'], in_tm
    index_name = 'out_시각'
    start_time = file_contents.iloc[0][index_name]    
    start_time = datetime.strptime(start_time,'%Y-%m-%d %H:%M:%S').strftime(UTC_Style)
    
    end_time = datetime.now().strftime(UTC_Style)
    # file 이름은 반드시 measurement name과 동일해야 한다.
    bk_name = folder_name
    ms_name = file_name.replace(".csv", "")       
    print("========= start_time :: ", start_time)    
        
    # (3) 데이터 삭제
    delete_api = InfluxClient.DBClient.delete_api()
    delete_api.delete(start_time, end_time, f'_measurement={ms_name}', bucket=bk_name, org = InfluxClient.influx_setting["org"])
    print("========== drop measurement : ",ms_name," ==========")

In [4]:
#################################################################
# 삭제할 measurement 데이터가 있는 사용자의 물리 경로 확인
#################################################################
def check_real_folder_and_delete_file(user_dir, folder_name) :    
    folder_path = os.path.join(user_dir, folder_name)
    file_name_list = os.listdir(folder_path)     

    # file name 은 반드시 measurement name과 동일해야 한다.
    for index, file_name in enumerate(file_name_list) :
        print("========= index :: ", index)  
        folder_name =  'air_outdoor_kweather' #folder name과 bucket이 동일하지 않을때 지정해준다
        drop_measurement_of_specific_time(folder_path, folder_name, file_name)
        
        
        
        

## 3. 함수 전달용 파라미터 초기화
3번 부분은 필요에 따라 수정해야 함.

In [5]:

"""
user_dir = '/home/jianso/바탕화면/folder/2023/inair_sliced/'
folder_name_list = [    
    'air_indoor_경로당'    
    , 'air_indoor_도서관'
    , 'air_indoor_아파트'
    , 'air_indoor_어린이집'
    , 'air_indoor_요양원'
    , 'air_indoor_유치원'
    , 'air_indoor_중학교'
    , 'air_indoor_체육시설'
]
------

"""
user_dir = '/home/jianso/바탕화면/folder/2023/3_outdoor/outdoor_sliced/'
folder_name_list = [
    'data_600'
    ,'data_650'
    
]


## 4. 함수 실행

In [6]:
for folder_name in folder_name_list :    
    print("===foldername :: ", folder_name, "===")
    check_real_folder_and_delete_file(user_dir, folder_name)


===foldername ::  data_600 ===
========= index ::  0
========= start_time ::  2022-05-01T00:00:00Z
========== drop measurement :  V02Q1940814  ==========
========= index ::  1
========= start_time ::  2022-05-01T00:00:00Z
========== drop measurement :  OC3CL200202  ==========
========= index ::  2
========= start_time ::  2022-05-01T00:00:00Z
========== drop measurement :  OC3CL200150  ==========
========= index ::  3
========= start_time ::  2022-05-01T00:00:00Z
========== drop measurement :  V02Q1940382  ==========
========= index ::  4
========= start_time ::  2022-05-01T00:01:00Z
========== drop measurement :  V02Q1940811  ==========
========= index ::  5
========= start_time ::  2022-05-01T00:00:00Z
========== drop measurement :  OC3CL200120  ==========
========= index ::  6
========= start_time ::  2022-04-30T23:59:00Z
========== drop measurement :  V02Q1940343  ==========
========= index ::  7
========= start_time ::  2022-05-01T00:01:00Z
========== drop measurement :  V02Q19401

========= start_time ::  2022-05-01T00:00:00Z
========== drop measurement :  OC3CL200122  ==========
========= index ::  17
========= start_time ::  2022-05-01T00:00:00Z
========== drop measurement :  V02Q1940529  ==========
========= index ::  18
========= start_time ::  2022-05-01T00:01:00Z
========== drop measurement :  OC3CL200215  ==========
========= index ::  19
========= start_time ::  2022-05-01T00:00:00Z
========== drop measurement :  OC3CL200106  ==========
========= index ::  20
========= start_time ::  2022-05-01T00:00:00Z
========== drop measurement :  OC3CL200014  ==========
========= index ::  21
========= start_time ::  2022-05-01T00:01:00Z
========== drop measurement :  V02Q1940631  ==========
========= index ::  22
========= start_time ::  2022-05-01T00:01:00Z
========== drop measurement :  V02Q1940625  ==========
========= index ::  23
========= start_time ::  2022-05-01T00:01:00Z
========== drop measurement :  V02Q1940859  ==========
========= index ::  24
========